### **Inflation bond**  

* [CPI data: 統計で見る日本 >> 中分類指数(全国) 月次 EXCEL閲覧用 >> 生鮮食品を除く総合](https://www.e-stat.go.jp/stat-search/files?page=1&layout=datalist&toukei=00200573&tstat=000001150147&cycle=1&year=20250&month=24101211&tclass1=000001150149&tclass2val=0)
* [財務省 物価連動国債](https://www.mof.go.jp/jgbs/topics/bond/10year_inflation-indexed/index.htm)

In [1]:
from myABBR import * ; import myUtil as mu 

jb378 = mu.makeJGB(jDT(2025,9,20),jDT(2035,3,20),0.014)
trdDT = jDT(2026,1,5)
setEvDT(trdDT) ; print(f'EvDT({getEvDT().ISO()}', end=') ')
print(f'jpnCmpYld: {jb378.JapanCompoundYield(94.56):.5%}', end=', ')
print( 'stdCmpYld: {:.5%}'.format(
               jb378.bondYield(cP(94.56),dcA365n, cmpdCMP, freqSA)))

EvDT(2026-01-05) jpnCmpYld: 2.05190%, stdCmpYld: 2.05179%


In [2]:
trdDT = jDT(2026,1,6) ;          stlDT = calJP.advance(trdDT,Tp1,DD)
setEvDT(trdDT) ; print(f'EvDT({getEvDT().ISO()}', end=') ')
print(f'jpnCmpYld: {jb378.JapanCompoundYield(94.56):.5%}', end=', ')
print( 'stdCmpYld: {:.5%}'.format(
               jb378.bondYield(cP(94.56),dcA365n, cmpdCMP, freqSA)))

EvDT(2026-01-06) jpnCmpYld: 2.05209%, stdCmpYld: 2.05197%


In [3]:
# BEI計算: 初期値  (effDT:前回利払日   nmlYLD: ベンチマーク国債利回り)
effDT,               matDT,       cpnRT,   rBndPRC,  nmlYLD         =\
jDT(2025,9,10), jDT(2035,3,10), 0.005/100,  97.55,  2.052/100 

# real bond object
rBndOBJ = mu.makeJGB(effDT,matDT,cpnRT)
# real yield
realYLD = rD(rBndOBJ.bondYield(cP(rBndPRC), dcA365n, cmpdCMP, freqSA),5)
beiRT   = nmlYLD -realYLD
print(f'real YLD:{realYLD:.5%}, (BEI)Break-even Infl.: {beiRT:.5%}') 

real YLD:0.27500%, (BEI)Break-even Infl.: 1.77700%


In [4]:
# CPIデータ、日付の調整
issCPI,       stCPI,          enCPI,         =\
109.60,  (2025,9,111.40), (2025,10,112.10)

aTrdDT,       aStlDT                         =\
trdDT-ds9,  stlDT-ds9;     setEvDT(aTrdDT)

# CPI指数オブジェクト設定と関連数値の計算
infCvHDL = ql.RelinkableZeroInflationTermStructureHandle() 
cpiIX    = ql.ZeroInflationIndex("Japan CPI", 
                        jpRegion, reviseF, freqM, lag0M, jpyFX, infCvHDL)
for yy, mm, xx in [stCPI, enCPI]: cpiIX.addFixing( jDT(yy,mm,1), xx )
enCpiDT = cpiIX.lastFixingDate()
refCPI  = round( ql.CPI.laggedFixing(cpiIX, aStlDT, lag3M, cpiLNR ),3)
ixRTO   = round( refCPI / issCPI , 5)
print(f'lastFixing DT:{enCpiDT.ISO()}, ' 
                            f'refCPI:{refCPI:.8f}, ixRatio:{ixRTO:.8f}')

lastFixing DT:2025-10-01, refCPI:112.03200000, ixRatio:1.02219000


In [5]:
# インフレカーブ                             (BEIでフラットカーブを設定)
infCvOBJ = ql.ZeroInflationCurve( aStlDT,
            [enCpiDT,enCpiDT+pD('10y')], [beiRT,beiRT], freqM, dcA365) #dcA365n
infCvHDL.linkTo(infCvOBJ)
# カーブ表示
dfInfCv  = pd.DataFrame( infCvOBJ.nodes(),columns=["node", "rate"] )
dfInfCv.node = dfInfCv.node.map(lambda x: x.ISO())
dfInfCv[:5].style.format({"rate": "{:.4%}"})

,node,rate
0,2025-10-01,1.7770%
1,2035-10-01,1.7770%


In [6]:
# inf bond object    >> スケジュールをds9で調整
iScdOBJ = ql.Schedule(effDT-ds9,matDT-ds9,pdFreqSA,calNL,unADJ,unADJ,dtGENb,EoMf)
iBndOBJ = ql.CPIBond (Tp1, parAMT, gwOLY, issCPI, lag3M, cpiIX, 
                                        cpiLNR, iScdOBJ, [cpnRT], dcA365n, unADJ)
# discount curve
dsCvOBJ, dsCvHDL = mu.ffTSH(aStlDT, nmlYLD, dcA365n, cmpdCMP, freqSA)
iBndOBJ.setPricingEngine(ql.DiscountingBondEngine(dsCvHDL))
# pricing
clnPRC  = iBndOBJ.cleanPrice()  ;        rClnPRC = clnPRC / ixRTO
accAMT  = iBndOBJ.accruedAmount()
infYLD  = iBndOBJ.bondYield(cP(clnPRC),  dcA365n, cmpdCMP, freqSA)
print(f'clnPRC:{clnPRC:.5f},  '      f'accAMT:{accAMT:.7f},  '
  f'nominalYLD:{infYLD:.7%},  '  f'realClnPRC:{rClnPRC:.7f} ')

clnPRC:99.73804,  accAMT:0.0016663,  nominalYLD:2.0520002%,  realClnPRC:97.5728931 


In [7]:
# cash flows
dfBND = mu.cpiBondCashFlow(iBndOBJ, yts=dsCvOBJ, past=1)
display( dfBND.style.format(fmtFUT) )
HCclnPRC = (dfBND.amount *dfBND.DF).sum() - accAMT
print(f'(hc) clnPRC:{HCclnPRC:.5f}')

,payDate,accruStart,accruEnd,cpi,coupon,amount,DF
0,2025-09-01,nan,nan,nan,nan%,nan,1.000000
1,2026-03-01,2025-09-01,2026-03-01,112.430475,0.0051%,0.0025,0.996538
2,2026-09-01,2026-03-01,2026-09-01,113.422284,0.0052%,0.0026,0.986335
3,2027-03-01,2026-09-01,2027-03-01,114.428365,0.0052%,0.0026,0.976400
4,2027-09-01,2027-03-01,2027-09-01,115.437798,0.0053%,0.0027,0.966402
5,2028-03-01,2027-09-01,2028-03-01,116.461757,0.0053%,0.0026,0.956668
6,2028-09-01,2028-03-01,2028-09-01,117.494798,0.0054%,0.0027,0.946873
7,2029-03-01,2028-09-01,2029-03-01,118.537002,0.0054%,0.0027,0.937335
8,2029-09-01,2029-03-01,2029-09-01,119.582681,0.0055%,0.0028,0.927738
9,2030-03-01,2029-09-01,2030-03-01,120.643405,0.0055%,0.0027,0.918393


(hc) clnPRC:99.73804


* 出力が見にくくなるため、コメントアウトした

In [8]:
# # 日付を10日に戻す (dfBNDのコピーでの処理)
# dfBND2 = dfBND.copy()
# for cc in ['payDate', 'accruStart', 'accruEnd'] :
#   dfBND2[cc] = pd.to_datetime(dfBND2[cc]) + pd.Timedelta(days=ds9)
#   dfBND2[cc] = dfBND2[cc].dt.strftime("%Y-%m-%d")
# dfBND2.style.format(fmtFUT)

In [9]:
# 年2回複利の計算
cfLeg = ql.Leg() 
for iso, amt in zip(dfBND.payDate[1:], dfBND.amount[1:]):
          cfLeg.append(ql.SimpleCashFlow(amt, iDT(iso)))
yldSA = ql.CashFlows.yieldRate(
          cfLeg, clnPRC+accAMT, dcA365n, cmpdCMP, freqSA, False)
print(f'semi-annual yield:{yldSA:.6%}')

semi-annual yield:2.051384%


In [10]:
[ (lg.date(), lg.amount()) for lg in cfLeg ][-3:]

[(Date(1,9,2034), 0.0030034601755009336),
 (Date(1,3,2035), 0.002980697691933767),
 (Date(1,3,2035), 120.21598426031213)]

* インフレ債 市場価格となるインフレ率の算出

In [14]:
# find implied inflation  infRT:implied インフレ率
def infSLVR(iRT):
  # Flat inf-curve
  infCvOBJ = ql.ZeroInflationCurve( aStlDT,
              [enCpiDT,enCpiDT+pD('10y')], [iRT,iRT], freqM, dcA365)
  infCvHDL.linkTo(infCvOBJ)
  # pricing
  clnPRC = iBndOBJ.cleanPrice() ;      rClnPRC = clnPRC / ixRTO
  return rBndPRC - rClnPRC
                               # accuracy   guess xMin  xMax
infRT = ql.Brent().solve(infSLVR,  1e-8,      0,  -0.1, 0.5)
print(f'real bond価格(rBndPRC):{rBndPRC:.5f}での' 
      f'インフレーション率(infRT):{infRT:.5%}'              )

real bond価格(rBndPRC):97.55000でのインフレーション率(infRT):1.77440%
